This is the updated notebook provided by AEMDA

In [ ]:
import pandas as pd
import numpy as np
from copy import copy
from itertools import combinations
import json
import yapf.yapflib.yapf_api

In [2]:
def isnan(v):
    """Check if a value is not a number of empty value"""
    return not v or str(v)==str(np.nan)

def get_all_combinations(columns, mandatory_columns=[], min_count=1):
    """
    if mandatory columns are included all returned combinations must include the columns
    """
    possible_permutations=[]
    for i in range(len(columns), min_count-1, -1):
        options = list(combinations(columns, i))
        if mandatory_columns:
            # Add the mandatory columns to every possible combination returned
            options = [list(set(mandatory_columns)|set(o)) for o in options]
        possible_permutations=possible_permutations+ options
    final_possible_permutations =  possible_permutations+get_all_combinations(mandatory_columns, min_count=2) if mandatory_columns else possible_permutations
    return final_possible_permutations

def clean_dataset(df):
    columns = df.columns
    for column in columns:
        if df[column].dtype == 'O':
            df[column] = df[column].str.strip()
    return df
# Needs to find another source for average economical life of vehicles
aveEconLife_df=clean_dataset(pd.read_csv('./giz/data/aveEconLife.csv'))
infras_fleet_df=clean_dataset(pd.read_csv('./giz/data/infras_fleet.csv', thousands=','))
age_df=clean_dataset(pd.read_csv('./giz/data/age_distribution.csv'))
trigger_fuel_consumption_df = clean_dataset(pd.read_csv('./giz/data/trigger_fuel_consumption.csv'))
infras_fleet_df.head()

,Vehicle category,Segment,Size Class,Technology,Fleet 2015,Fleet 2017,Note
0,Passenger cars,"PC petrol <1,4L","<1,4L",petrol (4S),395252.0,465065.0,NaN
1,Passenger cars,"PC petrol 1,4-<2L","1,4-<2L",petrol (4S),95260.0,112875.0,NaN
2,Passenger cars,PC petrol >=2L,>=2L,petrol (4S),8068.0,9631.0,NaN
3,Passenger cars,"PC diesel <1,4L","<1,4L",diesel,14285.0,16430.0,NaN
4,Passenger cars,"PC diesel 1,4-<2L","1,4-<2L",diesel,11843.0,13875.0,NaN


In [3]:
age_df.head()

,Vehicle category,Segment,note,Age,0,1,2,3,4,5,...,23,24,25,26,27,28,29,30,31,32
0,Passenger cars,"PC petrol <1,4L",NaN,%,0,0,0,0,0,1,...,1,1,1,0,0,0,0,0,0,0
1,Passenger cars,"PC petrol 1,4-<2L",NaN,%,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,Passenger cars,PC petrol >=2L,NaN,%,1,3,4,2,1,2,...,0,0,0,0,0,0,0,0,0,0
3,Passenger cars,"PC diesel <1,4L",NaN,%,0,0,0,0,0,0,...,0,3,10,3,3,2,0,1,0,1
4,Passenger cars,"PC diesel 1,4-<2L",NaN,%,0,1,1,1,2,1,...,1,1,1,3,0,0,4,1,0,1


In [4]:
trigger_fuel_consumption_df.head()

,ID Subsegment,Vehicle category,Sub-category (EMAP EEA),Segment,Fuel type in this tool,Technology (EMAP EEA),Technology in this tool,FC (g/km),"FC(l/100km), kg for CNG",Source,ID Vehicle cat,ID Fuel,ID Size,ID Technology,Unnamed: 14,note
0,PG10,Passenger cars,Gasoline 0.8-1.4 l,"PC petrol <1,4L",Motor Gasoline,PRE-ECE to open loop,PRE-ECE,57.69,7.8,"Infras,2018",P,G,1,0,NaN,NaN
1,PG11,Passenger cars,Gasoline 0.8-1.4 l,"PC petrol <1,4L",Motor Gasoline,Euro 1 and later,Euro 1,57.69,7.8,"Infras,2018",P,G,1,1,NaN,NaN
2,PG12,Passenger cars,Gasoline 0.8-1.4 l,"PC petrol <1,4L",Motor Gasoline,Euro 1 and later,Euro 2,57.69,7.8,"Infras,2018",P,G,1,2,NaN,NaN
3,PG13,Passenger cars,Gasoline 0.8-1.4 l,"PC petrol <1,4L",Motor Gasoline,Euro 1 and later,Euro 3,57.69,7.8,"Infras,2018",P,G,1,3,NaN,NaN
4,PG14,Passenger cars,Gasoline 0.8-1.4 l,"PC petrol <1,4L",Motor Gasoline,Euro 1 and later,Euro 4,57.69,7.8,"Infras,2018",P,G,1,4,NaN,NaN


In [5]:
def get_query(row, columns, equal_sign='=='):
    q=[]
    for o in columns:
        value = row[o]
        try:
            value = int(value)
            q.append(f'`{o}`=={value}')
        except Exception as e:
            if not isnan(value):
                value = f'"{value}"'
                q.append(f'`{o}`{equal_sign}{value}')
            else:
                q.append(f"`{o}`.isnull()")
    return q
    
def find_match(row, df2, on=[], exclude_query=[]):
    q=' & '.join(get_query(row, on))
    if exclude_query:
        q=f"{q} & not {exclude_query}"
    matched=df2.query(q) if q else []
    return matched
    
def merge_df(df1, df2, on=[], override_columns=False, force_match=False, if_not_matched_alt_columns=[], unique_column=None, mandatory_columns=[]):
    """
    merges two dataframe, 
    for every row of df1 it searches and picks first row on df2 whose 'on' columns matches
    """
    df1=df1.copy()
    df2=df2.copy()
    new_df=df1.copy()
    columns=df2.columns
    df1_columns=df1.columns
    selected_query=[]
    
    if type(on) is str:
        on=[on]
    all_columns_combinations=get_all_combinations(on, mandatory_columns=mandatory_columns) if force_match else [on]
    for idx, row in df1.iterrows():
        # clone the fields to be matched so it possible to pop them later
        matched=[]
        exclude_query=None
        is_matched = False
        selected_columns = []
        if selected_query:
            exclude_query = f"`{unique_column}` in {selected_query}"
                
        for column_combination in all_columns_combinations:
            selected_columns=column_combination
            matched=find_match(row, df2, on=column_combination, exclude_query=exclude_query)
            is_matched = len(matched)
            if is_matched:
                break
                
        if not is_matched and if_not_matched_alt_columns:
            # if no matches are found whatsoever find using alternative columns and do not exclude matched columns
            matched=find_match(row, df2, on=if_not_matched_alt_columns)
            is_matched = len(matched)
        if is_matched:
            matched_row=matched.iloc[0]
            if unique_column:
                selected_query.append(matched_row[unique_column])
            for column in matched.columns:
                if not column in df1_columns or override_columns:
                    df1.loc[idx, [column]] = [matched_row[column]]
            if force_match:
                # write a note for forced matching of fields identifying all fields used to get the match
                df1.loc[idx, ['note']] = [f"{len(selected_columns)} fields matched, {', '.join(selected_columns)}"]
    return df1

def to_int(d):
    """A quick way to convert a dataframe or list of strings into integers"""
    return list(map(lambda x: int(x.replace(',', '')) if type(x) is str else 0, list(d)))

## First open the excel file with the sample technology table from TEAM
Also make sure to load all definition of keys on the technology table

In [6]:
tech_df=pd.read_excel('sample_data/team_kenya.xlsx', engine='openpyxl')
# load definition of terms from the technology table
sample_df={}
sample_tech_df = pd.read_excel('sample_data/team_kenya.xlsx', engine='openpyxl')
for sheet_name in sample_tech_df.columns:
    try:
        sample_df[sheet_name]=pd.read_excel('sample_data/team_kenya.xlsx', engine='openpyxl', sheet_name=sheet_name)
    except Exception as e:
        pass

In [7]:
print('The list of indexed tables i.e the definitions of techology table attributes', '\n- '.join(sample_df.keys()))


The list of indexed tables i.e the definitions of techology table attributes ModeID
- VehTypeID
- MassCatID
- FuelID
- EngineID
- TransTypeID
- HybridFlag
- SecondHandImportFlag


In [8]:
print('preview sample data')
sample_df[list(sample_df.keys())[0]]

preview sample data


,ModeID,ModeNA
0,1,Road
1,2,Rail
2,3,Water
3,4,Air


In [9]:
# Merge techology table with its definition of terms to form one database
merged_df=tech_df.copy()
for column in sample_df.keys():
    merged_df=merge_df(
        merged_df, sample_df[column], on=[column]
    )


In [10]:
merged_df.head()

,TechID,ModeID,VehTypeID,MassCatID,FuelID,EngineID,TransTypeID,Availability,Final_Year,HybridFlag,...,FuelCondition,DenomUnit,FuelDensity_kg,FuelCalValue_MJ,EngineAB,EngineNA,TransTypeAB,TransTypeNA,HybridFlagDescription,SecondHandImportFlagDescription
0,10,1,2,4,1,10,1,1980,2000.0,0,...,NaN,l,0.745,32.1840,2TSTD1,2-Stroke Standard 1,P,Passenger Transport,Not hybrid,Not second hand import
1,20,1,2,4,12,10,1,1995,2010.0,0,...,NaN,Wh,NaN,0.0036,2TSTD1,2-Stroke Standard 1,P,Passenger Transport,Not hybrid,Not second hand import
2,25,1,2,4,12,11,1,2010,2020.0,0,...,NaN,Wh,NaN,0.0036,2TSTD2,2-Stroke Standard 2,P,Passenger Transport,Not hybrid,Not second hand import
3,26,1,2,4,12,12,1,2020,NaN,0,...,NaN,Wh,NaN,0.0036,2TECO,2-Stroke Economic,P,Passenger Transport,Not hybrid,Not second hand import
4,30,1,2,4,1,11,1,2000,2010.0,0,...,NaN,l,0.745,32.1840,2TSTD2,2-Stroke Standard 2,P,Passenger Transport,Not hybrid,Not second hand import


In [11]:
# preview all the columns on the new merged dataframe
print('-', '\n- '.join(merged_df.columns))

- TechID
- ModeID
- VehTypeID
- MassCatID
- FuelID
- EngineID
- TransTypeID
- Availability
- Final_Year
- HybridFlag
- spec_energ_consump
- spec_energ_electric
- noise_class
- PurchasePrice
- ExpectedLifeFactor
- Capacity
- AveEconLife
- SecondHandImportFlag
- ModeNA
- VehTypeAB
- VehTypeNA
- MassCatNA
- FuelTypeID
- FuelAB
- FuelNA
- FuelCondition
- DenomUnit
- FuelDensity_kg
- FuelCalValue_MJ
- EngineAB
- EngineNA
- TransTypeAB
- TransTypeNA
- HybridFlagDescription
- SecondHandImportFlagDescription


In [12]:
# combine fleet, aveEconLife_df and age distribution data
infras_fleet_n_aveEconLife_df =  merge_df(infras_fleet_df, aveEconLife_df, on=['Vehicle category'])
fleet_n_age = merge_df(infras_fleet_n_aveEconLife_df, age_df, on=['Segment'])

In [13]:
fleet_n_age.head()

,Vehicle category,Segment,Size Class,Technology,Fleet 2015,Fleet 2017,Note,Median life expectancy,note,Age,...,23,24,25,26,27,28,29,30,31,32
0,Passenger cars,"PC petrol <1,4L","<1,4L",petrol (4S),395252.0,465065.0,NaN,14.0,NaN,%,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Passenger cars,"PC petrol 1,4-<2L","1,4-<2L",petrol (4S),95260.0,112875.0,NaN,14.0,NaN,%,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Passenger cars,PC petrol >=2L,>=2L,petrol (4S),8068.0,9631.0,NaN,14.0,NaN,%,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Passenger cars,"PC diesel <1,4L","<1,4L",diesel,14285.0,16430.0,NaN,14.0,NaN,%,...,0.0,3.0,10.0,3.0,3.0,2.0,0.0,1.0,0.0,1.0
4,Passenger cars,"PC diesel 1,4-<2L","1,4-<2L",diesel,11843.0,13875.0,NaN,14.0,NaN,%,...,1.0,1.0,1.0,3.0,0.0,0.0,4.0,1.0,0.0,1.0


In [14]:
total_fleet_2015={}
for year in ['2015', '2017']:
    data=fleet_n_age[f'Fleet {year}']
    s=data.sum()
    if year=='2015':
        total_fleet_2015=data
    print(f'Total fleet in {year} is {s}')
    

Total fleet in 2015 is 1360937.0
Total fleet in 2017 is 1723347.0


In [15]:
c_df=merge_df(trigger_fuel_consumption_df, fleet_n_age, on=['Vehicle category', 'Segment'])

preview fleet and age distribution for different technologies

In [16]:
fleet_n_age.head()

,Vehicle category,Segment,Size Class,Technology,Fleet 2015,Fleet 2017,Note,Median life expectancy,note,Age,...,23,24,25,26,27,28,29,30,31,32
0,Passenger cars,"PC petrol <1,4L","<1,4L",petrol (4S),395252.0,465065.0,NaN,14.0,NaN,%,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Passenger cars,"PC petrol 1,4-<2L","1,4-<2L",petrol (4S),95260.0,112875.0,NaN,14.0,NaN,%,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Passenger cars,PC petrol >=2L,>=2L,petrol (4S),8068.0,9631.0,NaN,14.0,NaN,%,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Passenger cars,"PC diesel <1,4L","<1,4L",diesel,14285.0,16430.0,NaN,14.0,NaN,%,...,0.0,3.0,10.0,3.0,3.0,2.0,0.0,1.0,0.0,1.0
4,Passenger cars,"PC diesel 1,4-<2L","1,4-<2L",diesel,11843.0,13875.0,NaN,14.0,NaN,%,...,1.0,1.0,1.0,3.0,0.0,0.0,4.0,1.0,0.0,1.0


Get the year of availability from the 'Technology in this tool' column from GIZ data

https://www.rac.co.uk/drive/advice/emissions/euro-emissions-standards/ is used as source

## Vehicle import assumptions
1. 85% of cars, minibuses and trucks in the base year are second-hand imports

2. Everything else is not second-hand imports (i.e. brought into, or manufactured in, Kenya as a new vehicle)

In [17]:
tech_availability={
    "EURO1": 1992,
    "EURO2": 1997,
    "EURO3": 2001,
    "EURO4": 2006,
    "EURO5": 2011,
    "EURO6": 2015,
}
# other assumptions
KENYA_TYPICAL_V_IMPORT_AGE=8
kenya_new_used_car_ratio={
 'Average moto':[1, 0],
 'Coach (bus)':[1, 0],
 'Urban (bus)':[1, 0], 
    
 'Mini (bus)':[0.15, 0.85],
    
 'Heavy (truck)':[0.15, 0.85],
 'Large (car)':[0.15, 0.85],
 'Medium (car)':[0.15, 0.85],
 'Medium (truck)':[0.15, 0.85],
 'PanelSideVan (truck)':[0.15, 0.85],
 'Small (car)':[0.15, 0.85],

}
mandatory_tech_id_columns = ['MassCatNA', 'VehTypeNA', 'FuelAB']

Create some helper functions for transforming GIZ data to TEAM technology data. Assumptions and comments are included on each function

For vehicle stock assume vehicle count is divided equally among different vehicle segment for the same vehicle category e.g:
- PC petrol <1,4L
- PC petrol 1,4-<2L
- PC petrol >=2L
- PC diesel <1,4L
- PC diesel 1,4-<2L
- PC diesel >=2L
- LCV petrol M+N1-I
- LCV petrol N1-I
- LCV petrol N1-II
- LCV petrol N1-III
- LCV diesel N1
- ...

In [18]:
def getValueByMapping(value_to_match, mapping):
    """
    @value_to_match: value from GIZ data
    @mapping: A dictionary {
        key: TEAM Techology value
        value: List of GIZ data attributes which should be mapped to the TEAM tech value
    }
    """
    for k, v in mapping.items():
        if value_to_match in v:
            return k
    return value_to_match

def VehTypeID(row):
    vehicle_category=row['Vehicle category']
    return {
        "Light commercial vehicles":"Car",
        "Heavy goods vehicles":"Truck",
        "Buses":"Bus",
        "Motorcycles":"Motorcycle",
        'Passenger cars': "Car",
        'Mopeds': 'Motorcycle',
    }.get(vehicle_category, vehicle_category)
    
    
def MassCatID(row):
    """
    The mass categories were classified by considering GIZ data engine size class
    GIZ Sub-category (EMAP EEA) is also used for the classification 
    for technologies which do not have Size Class
    
    - minibuses (vehtypeID = 4 & masscatID == 11) 
        would represent 14-seater matatus (Toyota Hi-Ace etc., imported as 8 yr old vehicles); 
    - urban buses (vehtypeID == 4 & masscatID == 12) 
        would represent 33-seater matatus (built new on Isuzu truck frames); 
    - coaches (vehtypeID == 4 & masscatID == 13) 
        would represent the bigger intercity buses (e.g. the service that runs from Nairobi to Mombasa…)
 
    """
    size_class = row['Size Class']
    mapping = {
        'Small (car)':['<1,4L', 'Diesel < 1.4 l','Hybrid gasoline <1.4 l', 'Gasoline 0.8-1.4 l','Hybrid gasoline <1.4 l', 'LPG', 'CNG'],
        'Medium (car)':['1,4-<2L','Diesel < 1.4–2.0 l', 'Gasoline 1.4–2.0 l', 'Hybrid gasoline 1.4‚<2.0 l', 'Hybrid gasoline 1.4–2.0 l'],
        'Large (car)':['>=2L', 'Diesel > 2.0 l','Gasoline > 2.0 l', 'Hybrid gasoline >2.0 l', 'Motor Gasoline_3- > 1760 Kg'],
        'Average moto':['<=150cc',' <=150cc', '151-250cc', '251-750cc', 
                        '2-stroke < 50 cm³', '4-stroke < 50 cm³',
                        '2-stroke > 50 cm³',
                        '4-stroke 250–750 cm³',
                        '4-stroke < 250 cm³',
                        '4-stroke > 750 cm³',
                       
        ],
        'Mini (bus)':['M+N1-I', 'M+N1-II', 'M+N1-III', 'Urban buses, standard 3.5-15 t'],
        'Urban (bus)':[
                'Urban CNG buses',
                'Urban buses, standard 15–18 t',
        ],
        'Coach (bus)':['<=18t', 'Coaches, standard <=18 t',],
        'PanelSideVan (truck)':[
            'Diesel_2- 1305-1760 Kg',
            'Diesel_1- <1305 Kg',
            'Motor Gasoline_2- 1305-1760 Kg',
            'Motor Gasoline_1- <1305 Kg', 
            'Diesel_3- > 1760 Kg',
            'N1', 'Gasoline < 3.5 t', 'Diesel < 3.5 t','N1-I', 'N1-II', 'N1-III', 'LCV diesel N1-III'],
        'Medium (truck)':[ 
            '12-14 t',
            '14-20 t',
            'Gasoline > 3.5 t',
            'Diesel > 3.5 t',
            'RT<=7,5t',
            '<=7.5 t',
            '7.5-12 t',
            'RT >7,5-12t', 
            'RT >12-14t', 
            'RT >14-20t',  
            'RT >20-26t',  
        ],
        'Heavy (truck)':[
            'TT/AT >20-28t', 
            'TT/AT >28-34t', 
            'TT/AT >34-40t',       
            '> 32 t','20-26 t',
            'TT/AT 20-28 t',
            'TT/AT 28-34 t',
            'TT/AT 34-40 t',
        ],
        'CarDerivedVan (truck)':[],
        'Pickup4x4Van (truck)':[],
        'DropAndTipVan (truck)':[],
        'BoxLutonInsVan (truck)':[],
        'OtherVan (truck)': []
    }
    v=getValueByMapping(size_class, mapping) 
    return  getValueByMapping(row['Sub-category (EMAP EEA)'], mapping) if isnan(v) else v

def FuelID(row):
    """
    For hybdrid cars, the fossil fuel is selected as Fuel type
    """
    mapping = {
        'GAS':['Motor Gasoline', 'Hybrid gasoline <1.4 l'],
        'DIESEL': ['Diesel Oil'],
        'GAS_IMP': [],
        'DIE_IMP': [],
        'LPG': ['LPG'],
        'MEOH': [],
        'E85': [],
        'B100': [],
        'CNG': ['CNG'],
        'CBG': [],
        'EL': [],
        'GH2': [],
        'LH2': [],
        'KEROSENE': [],
        'BIOJET': [],  
    }
    value = getValueByMapping(row['Fuel type in this tool'], mapping)
#     for hybrid cars use fossil fuel FUEL Type
    if not value or isnan(value):
        sub_category = row['Sub-category (EMAP EEA)'].lower()
        if 'gasoline' in sub_category:
            return 'GAS'
        if 'diesel' in sub_category:
            return 'DIESEL'
    return value

def EngineID(row):
    value =  str(row['Technology in this tool']).strip().replace(' ', '').upper()
    if value and 'PRE' in value:
        return 'CONV'
    return value

def Availability(row):
    engine_class = EngineID(row)
    return tech_availability.get(engine_class)

def TransTypeID(row):
    """
    Passenger cars 
    Light commercial vehicles
    Heavy goods vehicles
    Buses 
    Mopeds 
    Motorcycles 
    """
    mapping={
        "P":["Passenger cars", "Buses", "Mopeds", "Motorcycles"],
        "F":["Light commercial vehicles", "Heavy goods vehicles"],
        
    }
    return getValueByMapping(row['Vehicle category'], mapping)

In [19]:
model = {
    'ModeNA': lambda x: 'Road',
    'VehTypeNA':VehTypeID,
    'MassCatNA': MassCatID,
    'FuelAB': FuelID, #use originasl fossil fuel ID for hybdrid
    'EngineAB':EngineID,
    'TransTypeAB': TransTypeID,
    'HybridFlag': lambda x: 1 if 'Hybrid' in x['Sub-category (EMAP EEA)'] else 0,
    'spec_energ_consump': lambda x: x['FC(l/100km), kg for CNG'], #assume energy consumption the same for new and second hand 
    'ExpectedLifeFactor':lambda a: 1,
    'AveEconLife': lambda x: x['Median life expectancy'], #is the same for second and new car
    'Capacity':None, #leave blank for now
    'Availability': Availability, # use engine type from engine type + 8 years in Kenya
    'Final_Year': None, #     Ignore for now
    #'SecondHandImportFlag': None, #for vehicle stock use the 15% for new and 85 for second hand of the total stock
    'spec_energ_electric': None,
    'PurchasePrice': None, #Ignore for now
}

tech_database_data=[]
for idx, row in c_df.iterrows():
    segment = row['Segment']
    vehicle_category=row['Vehicle category']
    age_distribution={}
    for i in range(33):
        age_distribution[str(i)]=row[str(i)]
    for second_hand_flag in [0,1]:
        data={'Segment':segment, "Vehicle category":vehicle_category}
        data['SecondHandImportFlag'] = second_hand_flag
        for key, func in model.items():
            value = func
            if func:
                value=func(row)
            data[key]=value
            if key =='Availability' and data['SecondHandImportFlag']==1 and value:
                data[key]=value+KENYA_TYPICAL_V_IMPORT_AGE   
    
        data.update({ 
            'AgeDistr': age_distribution,
        })    
        tech_database_data.append(data)
new_tech_df=pd.DataFrame.from_dict(tech_database_data).drop_duplicates(
    list(tech_database_data[0].keys())[:18]    
)

In [20]:
for idx, row in new_tech_df.iterrows():
    second_hand_flag = row['SecondHandImportFlag']
    segment = row['Segment']
    vehicle_category=row['Vehicle category']
    matches_df = c_df.query(f"`Vehicle category`=='{vehicle_category}' & Segment =='{segment}'")
    fleet_2015 =0

    fleet_2015 = 2*matches_df['Fleet 2015'].mean()/(len(
        new_tech_df.query(f"`Vehicle category`=='{vehicle_category}' & Segment =='{segment}'")
    ) or 1)
    try:
        used_car_ratio = kenya_new_used_car_ratio.get(row.MassCatNA)
        NumVeh = fleet_2015*used_car_ratio[second_hand_flag]
        new_tech_df.loc[idx, ['NumVeh']] = [NumVeh] 
    except:
        print(vehicle_category, segment, row.MassCatNA)
        break


In [21]:
# check vehicle stock if calculated properly
for idx, row in new_tech_df.drop_duplicates(['Vehicle category', 'Segment']).iterrows():
    vehicle_category=row['Vehicle category']
    segment=row["Segment"]
    print(vehicle_category,segment, new_tech_df.query(f"`Vehicle category`=='{vehicle_category}' & Segment =='{segment}'")['NumVeh'].sum())

Passenger cars PC petrol <1,4L 395251.99999999994
Passenger cars PC petrol 1,4-<2L 95259.99999999999
Passenger cars PC petrol >=2L 8068.0
Passenger cars PC diesel <1,4L 14285.000000000002
Passenger cars PC diesel 1,4-<2L 11843.0
Passenger cars PC diesel >=2L 7697.999999999999
Passenger cars nan 0.0
Light commercial vehicles LCV petrol M+N1-I 4355.0
Light commercial vehicles LCV petrol N1-I 4355.0
Light commercial vehicles LCV petrol N1-II 7537.0
Light commercial vehicles LCV petrol N1-III 3212.0
Heavy goods vehicles LCV petrol N1-III 0.0
Light commercial vehicles LCV diesel N1 1675.0
Light commercial vehicles LCV diesel M+N1-I 1675.0
Light commercial vehicles LCV diesel M+N1-II 23197.0
Light commercial vehicles LCV diesel N1-II 23197.0
Light commercial vehicles LCV diesel M+N1-III 12506.999999999998
Heavy goods vehicles LCV diesel M+N1-III 12506.999999999998
Heavy goods vehicles LCV diesel N1-III 25013.999999999996
Heavy goods vehicles RigidTruck <7,5t 12448.0
Heavy goods vehicles Rigi

The fields below from the example technology table are used to match the GIZ data with already know TechID

In [22]:
# this task may take up to 30+ minutes so be patient
tech_id_matching_columns = {
    'SecondHandImportFlag':'SecondHandImportFlag',
    'ModeNA': 'ModeID', 
    
     'EngineAB':'EngineID', 
    'MassCatNA':'MassCatID',
    
    'VehTypeNA':'VehTypeID', 
    'FuelAB':'FuelID',
   
    'HybridFlag':'HybridFlag', 
    'TransTypeAB':'TransTypeID', 
    'Availability':'Availability', 
    'AveEconLife':'AveEconLife',
    'spec_energ_consump':'spec_energ_consump', 
#     The fields below were ignore
#     'ExpectedLifeFactor', 
#     'Capacity', 
#     'Final_Year', 
#     'spec_energ_electric', 
#     'PurchasePrice'
}
# most useful for not matching bodaboda data
if_not_matched_alt_columns =  [
            'VehTypeNA', 
               'MassCatNA', 
               'FuelAB'
        ]
merged_new_df=merge_df(new_tech_df, merged_df, if_not_matched_alt_columns=if_not_matched_alt_columns, on=list(tech_id_matching_columns.keys()), force_match=True, mandatory_columns = mandatory_tech_id_columns, unique_column='TechID')
merged_new_df.query('TechID.isnull()')

,Segment,Vehicle category,SecondHandImportFlag,ModeNA,VehTypeNA,MassCatNA,FuelAB,EngineAB,TransTypeAB,HybridFlag,...,FuelNA,FuelCondition,DenomUnit,FuelDensity_kg,FuelCalValue_MJ,EngineNA,TransTypeNA,HybridFlagDescription,SecondHandImportFlagDescription,note


In [23]:
merged_new_df.query('TechID.isnull()').to_csv('out_data/TechnologyNoTechID.csv')

In [24]:
merged_new_df_with_ids=merged_new_df.copy()
for key, value in tech_id_matching_columns.items():
    if key and value:
        try:
            merged_new_df_with_ids=merge_df(merged_new_df_with_ids, sample_df[value], on=[key], override_columns=True)
        except:
            pass

### Export data in TEAM format

In [25]:
team_columns=list(sample_tech_df.columns)+['note']

In [37]:
non_duplicate_technologies_df=(merged_new_df_with_ids[team_columns]
     .drop_duplicates(set(team_columns)-{'TechID'})
    
).drop_duplicates(['TechID'])
non_duplicate_technologies_df.to_csv('out_data/Technology.csv')
non_duplicate_technologies_df.query('~TechID.isnull()')

,TechID,ModeID,VehTypeID,MassCatID,FuelID,EngineID,TransTypeID,Availability,Final_Year,HybridFlag,spec_energ_consump,spec_energ_electric,noise_class,PurchasePrice,ExpectedLifeFactor,Capacity,AveEconLife,SecondHandImportFlag,note
0,70.0,1.0,3.0,1.0,1.0,1.0,1.0,NaN,None,0,7.8,None,0.0,None,1,None,14.0,0,"8 fields matched, VehTypeNA, MassCatNA, Hybrid..."
1,9070.0,1.0,3.0,1.0,1.0,1.0,1.0,NaN,None,0,7.8,None,0.0,None,1,None,14.0,1,"8 fields matched, VehTypeNA, MassCatNA, Hybrid..."
2,90.0,1.0,3.0,1.0,1.0,3.0,1.0,1992.0,None,0,7.8,None,0.0,None,1,None,14.0,0,"9 fields matched, VehTypeNA, MassCatNA, Hybrid..."
3,9090.0,1.0,3.0,1.0,1.0,3.0,1.0,2000.0,None,0,7.8,None,0.0,None,1,None,14.0,1,"9 fields matched, VehTypeNA, MassCatNA, Hybrid..."
4,100.0,1.0,3.0,1.0,1.0,4.0,1.0,1997.0,None,0,7.8,None,1.0,None,1,None,14.0,0,"8 fields matched, VehTypeNA, MassCatNA, Hybrid..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,20.0,1.0,2.0,4.0,1.0,3.0,1.0,2000.0,None,0,2.7,None,2.0,None,1,None,9.0,1,"2 fields matched, MassCatNA, VehTypeNA"
268,25.0,1.0,2.0,4.0,1.0,4.0,1.0,1997.0,None,0,2.7,None,2.0,None,1,None,9.0,0,"2 fields matched, MassCatNA, VehTypeNA"
269,26.0,1.0,2.0,4.0,1.0,4.0,1.0,2005.0,None,0,2.7,None,2.0,None,1,None,9.0,1,"2 fields matched, MassCatNA, VehTypeNA"
270,50.0,1.0,2.0,4.0,1.0,5.0,1.0,2001.0,None,0,2.7,None,2.0,None,1,None,9.0,0,"2 fields matched, MassCatNA, VehTypeNA"


### calculate vehicle stock by using the age distribution from the fleet age distribution table by GIZ

In [38]:
age_data=[]
for idx, row in merged_new_df_with_ids.drop_duplicates(set(team_columns)-{'TechID'}).iterrows():
    TotalNumVeh = row.NumVeh
    sumPercentage = sum(row.AgeDistr.values())
    for age, percentage in row.AgeDistr.items():
        numVeh = TotalNumVeh*percentage/sumPercentage
        data={
            "CountryID": 9,
            "TechID": row.TechID,
            "Age": age,
            "TransTypeID": row.TransTypeID,
            "VehTypeID": row.VehTypeID,
            "MassCatID": row.MassCatID,
            "EngineID": row.EngineID,
            "FuelID": row.FuelID,
            "NumVeh":numVeh, #int(numVeh) if not isnan(numVeh) else numVeh,
        }
        age_data.append(data)
vehicle_stock = pd.DataFrame.from_dict(age_data)
vehicle_stock.head()

,CountryID,TechID,Age,TransTypeID,VehTypeID,MassCatID,EngineID,FuelID,NumVeh
0,9,70.0,0,1.0,3.0,1.0,1.0,1.0,0.0
1,9,70.0,1,1.0,3.0,1.0,1.0,1.0,0.0
2,9,70.0,2,1.0,3.0,1.0,1.0,1.0,0.0
3,9,70.0,3,1.0,3.0,1.0,1.0,1.0,0.0
4,9,70.0,4,1.0,3.0,1.0,1.0,1.0,0.0


### only export the stock data with NumVeh>=0

In [39]:
vehicle_stock.query("NumVeh>=0.01").to_csv('out_data/VehicleStock.csv')

## Generate Vehicle Km (vkm) for different tech IDS

In [41]:
merged_new_df_with_ids.query('Segment.isnull()').drop_duplicates(['FuelAB'])

,Segment,Vehicle category,SecondHandImportFlag,ModeNA,VehTypeNA,MassCatNA,FuelAB,EngineAB,TransTypeAB,HybridFlag,...,FuelNA,FuelCondition,DenomUnit,FuelDensity_kg,FuelCalValue_MJ,EngineNA,TransTypeNA,HybridFlagDescription,SecondHandImportFlagDescription,note
84,NaN,Passenger cars,0,Road,Car,Small (car),LPG,CONV,P,0,...,Liquified Petroleum Gas,NaN,l,0.550,25.300,Conventional (pre-EURO),Passenger Transport,Not hybrid,Not second hand import,"7 fields matched, VehTypeNA, MassCatNA, Hybrid..."
98,NaN,Passenger cars,0,Road,Car,Small (car),CNG,EURO4,P,0,...,Compressed Natural Gas,200 bar (20 MPa),l,0.174,8.300,Euro 4 (2005-09),Passenger Transport,Not hybrid,Not second hand import,"8 fields matched, VehTypeNA, MassCatNA, Hybrid..."
104,NaN,Passenger cars,0,Road,Car,Small (car),GAS,EURO4,P,1,...,Gasoline,NaN,l,0.745,32.184,Euro 4 (2005-09),Passenger Transport,HEV,Not second hand import,"7 fields matched, VehTypeNA, MassCatNA, Hybrid..."


In [43]:
trigger_vkm_df=pd.read_csv('./giz/data/trigger2021_vKm.csv')
trigger_vkm_df.head()

,Categeory,Vehicle category,Transport mode,Size class,Segment,Fuel Type,Average VKT(km/year/vehicle),Average VKT Source,VKT note,Urban VKT Ratio,Highway VKT Ratio,Rural VKT Ratio,VKT Ratio note
0,Passenger car,Passenger cars,Passenger,1- <1400cm3,"PC petrol <1,4L",Motor Gasoline,23023.0,"INFRAS, 2018",NaN,0.5,0.25,0.25,There are more passenger cars in urban areas h...
1,Passenger car,Passenger cars,Passenger,1- <1400cm3,"PC diesel <1,4L",Diesel Oil,15448.0,"INFRAS, 2018",NaN,0.5,0.25,0.25,There are more passenger cars in urban areas h...
2,Passenger car,Passenger cars,Passenger,2- 1400-2000cm3,"PC petrol 1,4-<2L",Motor Gasoline,19014.0,"INFRAS, 2018",NaN,0.5,0.25,0.25,There are more passenger cars in urban areas h...
3,Passenger car,Passenger cars,Passenger,2- 1400-2000cm3,"PC diesel 1,4-<2L",Diesel Oil,13108.0,"INFRAS, 2018",NaN,0.5,0.25,0.25,There are more passenger cars in urban areas h...
4,Passenger car,Passenger cars,Passenger,3- >2000cm3,PC petrol >=2L,Motor Gasoline,25213.0,"INFRAS, 2018",NaN,0.5,0.25,0.25,There are more passenger cars in urban areas h...


In [44]:
trigger_fuel_consumption_df['Segment'].drop_duplicates()

0                     PC petrol <1,4L
7                   PC petrol 1,4-<2L
14                     PC petrol >=2L
21                    PC diesel <1,4L
28                  PC diesel 1,4-<2L
35                     PC diesel >=2L
42                                NaN
55                  LCV petrol M+N1-I
56                    LCV petrol N1-I
57                   LCV petrol N1-II
58                  LCV petrol N1-III
65                      LCV diesel N1
66                  LCV diesel M+N1-I
67                 LCV diesel M+N1-II
68                   LCV diesel N1-II
69                LCV diesel M+N1-III
76                  LCV diesel N1-III
77                   RigidTruck <7,5t
84                 RigidTruck 7,5-12t
91                 RigidTruck >12-14t
98                 RigidTruck >14-20t
100                RigidTruck >20-26t
101                     TT/AT >20-28t
102                     TT/AT >28-34t
103                     TT/AT >34-40t
105                   Coach Std <=18t
111    Urban

In [45]:
vkm_data=[]
journey_types={
    'Urban':1,
    'Rural':2,
    'Highway':3
}
for idx, row in merged_new_df_with_ids.drop_duplicates(set(team_columns)-{'TechID'}).iterrows():
    TotalNumVeh = row.NumVeh
    vehicle_category=row['Vehicle category']
    segment= row["Segment"]
    segment= f"Segment=='{segment}'" if not isnan(segment) else 'Segment.isnull()'
    matched_vkm =  trigger_vkm_df.query(f"`Vehicle category`=='{vehicle_category}' & {segment}").iloc[0]
#     if isnan(TotalNumVeh) or TotalNumVeh==0:
#         print(vehicle_category, segment, TotalNumVeh)
    for JSTypeVSMNA, JSTypeVSMID in journey_types.items():
        vkt_ratio = matched_vkm[f'{JSTypeVSMNA} VKT Ratio']
        vkm=TotalNumVeh*matched_vkm['Average VKT(km/year/vehicle)']*vkt_ratio
        data={
                'ScenarioID':1,
                'CountryID':9,
                'TechID': row.TechID,
                'JSTypeVSMID':JSTypeVSMID,
                'TechDescription': f"{row.ModeNA}-{row.MassCatNA}-{row.EngineNA}-{row.TransTypeNA}",
                'TransTypeID':row.TransTypeID,
                'VehTypeID':row.VehTypeID,
                'MassCatID':row.MassCatID,
                'VKM': vkm/1e9
        }
        vkm_data.append(data)
# .sort_values(['TechID'])
pd.DataFrame.from_dict(vkm_data).fillna(0).to_csv('out_data/vkmData.csv')